In [1]:
# import sys
# sys.path.append('./FeatBoost/')

In [1]:
import os
print(os.getcwd())
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer
import xgboost as xgb
import numpy as np
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import pandas as pd
from sklearn.linear_model import LogisticRegression
from FeatBoost.feat_selector import FeatureSelector
import json
import gc

/mnt/nfs/home/chhedah/hriday_branch/delirium_pipeline


In [2]:
gim_cohort = pd.read_parquet("./DataProcessing/Sep_24_gim_icd10.parquet")
sbk_gim = pd.read_parquet("./DataProcessing/Sep_24_sbk_gim_icd10.parquet")
non_gim_cohort = pd.read_parquet("./DataProcessing/Sep_24_non_gim_icd10.parquet")

zero_sum_columns = [col for col in gim_cohort.columns if gim_cohort[col].sum() == 0]
gim_cohort = gim_cohort.drop(columns=zero_sum_columns)
sbk_gim = sbk_gim.drop(columns=zero_sum_columns)
non_gim_cohort = non_gim_cohort.drop(columns=zero_sum_columns)

In [3]:
gim_cohort = gim_cohort.drop_duplicates()
gim_cohort = gim_cohort.reset_index(drop=True)
gim_cohort = pd.concat([gim_cohort, sbk_gim], ignore_index=True)

In [4]:
gim_cohort.shape

(4992, 1826)

In [5]:
# # Count NaN values in each column
# nan_counts = gim_cohort.isna().sum()

# # Filter to get only the columns that have at least one NaN value
# columns_with_nan = nan_counts[nan_counts > 0]/gim_cohort.shape[0]

# print(columns_with_nan)

In [6]:
# gim_cohort = gim_cohort.drop(columns=["diag_Disorientation unspecified", 
#                                       "diag_Oth/NOS symptom inv cogn fn & awareness", 
#                                       "lab_numeric_only_max_Albumin [Mass/volume] in Serum or Plasma", 
#                                       "lab_numeric_only_max_Cobalamin (Vitamin B12) [Moles/volume] in Serum or Plasma",
#                                       "lab_numeric_only_max_Creatinine [Moles/volume] in Serum or Plasma", 
#                                       "lab_numeric_only_max_Hemoglobin A1c [Mass/volume] in Blood",
#                                       "lab_numeric_only_max_Hemoglobin [Mass/volume] in Blood",
#                                       "lab_numeric_only_max_Leukocytes [#/volume] in Blood", 
#                                       "lab_numeric_only_max_Sodium [Moles/volume] in Serum or Plasma",
#                                       "lab_numeric_only_max_Thyrotropin [Units/volume] in Serum or Plasma"])

In [7]:
# lab_cols = ["lab_numeric_only_max_Albumin [Mass/volume] in Serum or Plasma",
#             "lab_numeric_only_max_Leukocytes [#/volume] in Blood", 
#             "lab_numeric_only_max_Sodium [Moles/volume] in Serum or Plasma",
#             "lab_numeric_only_max_Thyrotropin [Units/volume] in Serum or Plasma"]

# def impute(val):
#     if val > 0:
#         return 1.0
#     else:
#         return 0.0

# for lab in lab_cols:
#     print(lab)
#     gim_cohort[lab] = gim_cohort[lab].apply(impute)

In [8]:
# from sklearn.impute import KNNImputer
# X = X.drop(columns=["genc_id", "del_present"])


# # Initialize the KNNImputer with 3 neighbors.
# imputer = KNNImputer(n_neighbors=3)

# # Option 1: If all columns are numeric, impute the entire dataframe.
# imputed_full = pd.DataFrame(
#     imputer.fit_transform(gim_cohort),
#     columns=gim_cohort.columns,
#     index=gim_cohort.index
# )

# gim_cohort[lab_cols] = imputed_full[lab_cols]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=5323)

In [19]:
y = gim_cohort["del_present"]
X = gim_cohort

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=5323)

In [21]:
X_train.shape, X_test.shape

((4243, 1826), (749, 1826))

In [22]:
X_train = X_train.drop(columns=["genc_id", "del_present",  "diag_Disorientation unspecified", "diag_Oth/NOS symptom inv cogn fn & awareness", "diag_Ment/beh disrd dt alco use withdr w del", "diag_Palliative care"])
X_test = X_test.drop(columns=["genc_id", "del_present",  "diag_Disorientation unspecified", "diag_Oth/NOS symptom inv cogn fn & awareness", "diag_Ment/beh disrd dt alco use withdr w del", "diag_Palliative care"])

In [23]:
feature_names = list(X_train.columns)

In [24]:
X_train = np.array(X_train.values)
X_test = np.array(X_test.values)

y_train = np.array(y_train.values)
y_test = np.array(y_test.values)

In [25]:
X_train.shape[0] + X_test.shape[0]

4992

In [42]:
def apk_binary(y_true, y_pred_probs, k):
    """
    Computes the Average Precision at K (AP@K) for binary predictions.
    :param y_true: List or array of ground truth binary labels (0 or 1).
    :param y_pred_probs: List or array of predicted probabilities.
    :param k: The number of top predictions to consider.
    :return: Average Precision at K (AP@K).
    """
    # Sort predictions by predicted probability in descending order
    sorted_indices = np.argsort(y_pred_probs)[::-1]
    y_true_sorted = np.array(y_true)[sorted_indices]

    # Compute precision at each relevant position
    num_hits = 0.0
    score = 0.0

    for i in range(min(k, len(y_true_sorted))):
        if y_true_sorted[i] == 1:  # Relevant item
            num_hits += 1.0
            score += num_hits / (i + 1.0)  # Precision at position i+1

    # Normalize by the number of relevant items or k
    return score / min(sum(y_true), k) if sum(y_true) > 0 else 0.0

In [ ]:
def predict_normal(model, X_test):
    preds = model.predict(X_test)
    pred_probs = model.predict_proba(X_test)[:, 1]
    pred_probs_all = model.predict_proba(X_test)
    return preds, pred_probs, pred_probs_all

def calc_metrics(prediction, prediction_prob, labels, k=10):
    acc = accuracy_score(labels, prediction)
    f1 = f1_score(labels, prediction, average='binary')  # Use 'micro', 'macro', 'weighted' for multi-class
    precision = precision_score(labels, prediction, average='binary')
    recall = recall_score(labels, prediction, average='binary')
    roc_auc = roc_auc_score(labels, prediction_prob)
    
    # Precision@k calculation
    # Sort by prediction probabilities in descending order
    sorted_indices = np.argsort(prediction_prob)[::-1]
    top_k_indices = sorted_indices[:k]
    
    # Count true positives in the top k predictions
    top_k_labels = np.array(labels)[top_k_indices]
    precision_at_k = np.sum(top_k_labels) / k
    
    return acc, f1, precision, recall, roc_auc, precision_at_k
    # return acc, f1, precision, recall, roc_auc


def expected_calibration_error(samples, true_labels, M=10):
    # uniform binning approach with M number of bins
    bin_boundaries = np.linspace(0, 1, M + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]

    # get max probability per sample i
    confidences = np.max(samples, axis=1)
    # get predictions from confidences (positional in this case)
    predicted_label = np.argmax(samples, axis=1)

    # get a boolean list of correct/false predictions
    accuracies = predicted_label==true_labels

    ece = np.zeros(1)
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        # determine if sample is in bin m (between bin lower & upper)
        in_bin = np.logical_and(confidences > bin_lower.item(), confidences <= bin_upper.item())
        # can calculate the empirical probability of a sample falling into bin m: (|Bm|/n)
        prob_in_bin = in_bin.mean()

        if prob_in_bin.item() > 0:
            # get the accuracy of bin m: acc(Bm)
            accuracy_in_bin = accuracies[in_bin].mean()
            # get the average confidence of bin m: conf(Bm)
            avg_confidence_in_bin = confidences[in_bin].mean()
            # calculate |acc(Bm) - conf(Bm)| * (|Bm|/n) for bin m and add to the total ECE
            ece += np.abs(avg_confidence_in_bin - accuracy_in_bin) * prob_in_bin
    return ece[0]

def ace(samples, true_labels, bin_size=25):
    """
    Calculate the Adaptive Calibration Error (ACE) for a set of predictions.
    
    Args:
    samples (np.ndarray): Array of predicted probabilities for each class (shape: [num_samples, num_classes]).
    true_labels (np.ndarray): Array of true class labels.
    bin_size (int): Number of points in each bin.

    Returns:
    float: The calculated Adaptive Calibration Error (ACE).
    """
    num_samples = samples.shape[0]
    assert num_samples == len(true_labels), "Number of samples and true labels must match."

    # Get max probability per sample
    confidences = np.max(samples, axis=1)
    # confidences = samples[:, 1]

    # Get predictions from confidences
    predicted_label = np.argmax(samples, axis=1)

    # Get a boolean list of correct/false predictions
    accuracies = predicted_label == true_labels

    # Sort by confidence
    sorted_indices = np.argsort(confidences)
    sorted_confidences = confidences[sorted_indices]
    sorted_accuracies = accuracies[sorted_indices]

    ace = 0.0
    start_idx = 0
    
    mean_conf = []
    mean_acc = []
    while start_idx < num_samples:
        end_idx = min(start_idx + bin_size, num_samples)
        
        bin_confidences = sorted_confidences[start_idx:end_idx]
        bin_accuracies = sorted_accuracies[start_idx:end_idx]

        # Mean predicted probability in the bin
        mean_confidence = np.mean(bin_confidences)
        # Mean accuracy in the bin
        mean_accuracy = np.mean(bin_accuracies)
        
        mean_conf.append(mean_confidence)
        mean_acc.append(mean_accuracy)

        # Calculate bin contribution to ACE
        ace += np.abs(mean_confidence - mean_accuracy) * len(bin_confidences) / num_samples

        start_idx = end_idx

    return ace, mean_conf, mean_acc

def calc_metrics_at_thresholds(num_features, prediction_prob, labels, thresholds=[0.5], k=10):
    results = []
    
    # Calculate the ROC AUC once, as it does not depend on the threshold
    roc_auc = roc_auc_score(labels, prediction_prob)
    
    # Calculate Precision@k
    # Sort by prediction probabilities in descending order and take the top k predictions
    sorted_indices = np.argsort(prediction_prob)[::-1]
    top_k_indices = sorted_indices[:k]
    top_k_labels = np.array(labels)[top_k_indices]
    precision_at_k = np.sum(top_k_labels) / k

    for threshold in thresholds:
        # Apply the threshold to generate predictions
        prediction = (prediction_prob >= threshold).astype(int)
        
        # Calculate metrics
        acc = accuracy_score(labels, prediction)
        f1 = f1_score(labels, prediction, average='binary')
        precision = precision_score(labels, prediction, average='binary')
        recall = recall_score(labels, prediction, average='binary')
        map_k = apk_binary(labels, prediction_prob, k=labels.shape[0])
        
        # Calculate the percentage of positive predictions
        pred_positive_percentage = np.mean(prediction) 
        true_positive_percentage = np.mean(labels)
        
        # Append the results as a row to the list
        results.append({
            'num_features': num_features,
            'threshold': threshold,
            'accuracy': acc,
            'f1_score': f1,
            'precision': precision,
            'recall': recall,
            'roc_auc': roc_auc,
            'precision_at_k': precision_at_k,
            'map@k': map_k,
            'pred_positive_percentage': pred_positive_percentage,
            'true_positive_percentage': true_positive_percentage
        })
    
    return results

In [28]:
base = xgb.XGBClassifier(alpha= 1.0, reg_lambda= 2.0, learning_rate= 0.1, max_depth = 3, n_estimators = 200, scale_pos_weight = 10.0, subsample = 1.0)
est = xgb.XGBClassifier(alpha= 1.0, reg_lambda= 2.0, learning_rate= 0.1, max_depth = 3, n_estimators = 200, scale_pos_weight = 10.0, subsample = 1.0)

In [29]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [30]:
param_grid = {
    'n_estimators': [200],  # Number of boosting rounds
    'learning_rate': [0.1, 0.3, 0.05],  # Step size shrinkage
    'max_depth': [3, 5],  # Maximum depth of a tree
    'subsample': [1.0, 0.7],  # Subsample ratio of the training instances
    'colsample_bytree': [1.0, 0.7],  # Subsample ratio of columns
    'alpha': [1.0, 5.0],
    'reg_lambda': [2.0, 5.0],
    'scale_pos_weight': [10.0, 15.0]
}

# param_grid = {"alpha": 1.0, "lambda": 2.0, "learning_rate": 0.1, "max_depth": 3, "n_estimators": 200, "scale_pos_weight": 10.0, "subsample": 1.0}
featboost = FeatureSelector(base_estimator=base, 
                            estimator=est, 
                            feature_names=feature_names, 
                            metric='mapk', 
                            hyperparam_grid=param_grid, 
                            cv_folds=4, 
                            max_num_features=25, 
                            ranking_size=30, 
                            verbose=1)
y_train = y_train.astype(int)
featboost.fit(X_train, y_train)

Fitting 3 folds for each of 192 candidates, totalling 576 fits
[0.01585091, 0.013812742, 0.013331534, 0.013246621, 0.013115305, 0.012918431, 0.012422678, 0.012094134, 0.011741413, 0.011686749, 0.010874342, 0.010415033, 0.010234298, 0.010001167, 0.009928005, 0.009325542, 0.009171352, 0.009036911, 0.008874136, 0.00866046, 0.008573947, 0.008527317, 0.008424765, 0.00832518, 0.007375367, 0.007279444, 0.00725189, 0.0068105757, 0.006772844, 0.006581156]
Considering feature: diag_Urinary tract infection site not spec
CV-mapk: 0.24097653793809945


Considering feature: diag_Vomiting alone
CV-mapk: 0.24412713303738812


Considering feature: diag_While engaged in other spec activities
CV-mapk: 0.24097653793809945


Considering feature: diag_Oth abnormal result cv funct study
CV-mapk: 0.24097653793809945


Considering feature: diag_Chest pain unspecified
CV-mapk: 0.2460210650131983


Considering feature: diag_Ulcer low limb not elsewhere classified
CV-mapk: 0.24097653793809945


Considering featur

In [31]:
featboost_selected_features = featboost.selected_features
print(featboost_selected_features)

[0, 1279, 982, 414, 735, 1816, 1303, 389, 1223, 467, 399, 103, 946, 601, 358, 1297, 967, 927, 435, 1307, 387, 1640, 782, 1401, 1427]


In [32]:
featboost_selected_feature_names = [feature_names[x] for x in featboost_selected_features]
featboost_selected_feature_names

['age',
 'diag_Chest pain unspecified',
 'diag_Gastrointestinal haemorrhage NOS',
 'diag_Schizophrenia unspecified',
 'diag_Hypotension unspecified',
 'lab_numeric_only_max_Leukocytes [#/volume] in Blood',
 'diag_Ascites',
 'diag_Ment/beh disrd dt alco use withdrawal st',
 'diag_Urinary tract infection site not spec',
 "diag_Parkinson's disease",
 'diag_Ment/beh disrd dt use cocaine ac intox',
 'diag_Malignant neoplasm pancreas part unspec',
 'diag_Liver disease unspecified',
 'diag_Benign hypertension',
 'diag_Disorders of magnesium metabolism',
 'diag_Other and unspecified dysphagia',
 'diag_Acute pancreatitis unspecified',
 'diag_Alcoholic cirrhosis of liver',
 'diag_Anxiety disorder unspecified',
 'diag_Paraesthesia of skin',
 'diag_Ment/beh disrd dt harmful alcohol use',
 'diag_While engaged in other spec activities',
 'diag_COPD with acute exacerbation unspecified',
 'diag_Abnormal levels of other serum enzymes',
 'diag_Fracture of nasal bones closed']

In [33]:
results_folder = "./Sparse_Feb19/"

In [34]:
feature_file_path = results_folder+"feature_list.csv"
pd.DataFrame({"Features": featboost_selected_feature_names}).to_csv(feature_file_path, index=False)

In [35]:
feature_file_path = results_folder+"feature_list.csv"
featboost_selected_feature_names = list(pd.read_csv(feature_file_path)['Features'])

In [36]:
featboost_selected_features = []
for feat in featboost_selected_feature_names:
    featboost_selected_features.append(feature_names.index(feat))

In [37]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [38]:
def calculate_precision_at_k(prediction_prob, labels, max_k=50):
    precisions = []
    sorted_indices = np.argsort(prediction_prob)[::-1]  # Sort by predicted probabilities in descending order
    sorted_labels = np.array(labels)[sorted_indices]    # Sort true labels according to the sorted probabilities
    
    for k in range(1, max_k + 1):
        top_k_labels = sorted_labels[:k]
        precision_at_k = np.sum(top_k_labels) / k  # Precision@k = True Positives in top k / k
        precisions.append(precision_at_k)
        
    return precisions

def plot_precision_vs_k(prediction_prob, labels, file_name, max_k=50):
    # Calculate precision@k for each value of k up to max_k
    precisions = calculate_precision_at_k(prediction_prob, labels, max_k)
    
    # Plot Precision vs. k
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, max_k + 1), precisions, marker='o')
    plt.xlabel('K')
    plt.ylabel('Precision')
    plt.title('Precision vs. K')# (' + file_name + ')')
    plt.grid(True)
    
    # Set x-axis ticks at intervals of 10
    plt.xticks(range(0, max_k + 1, 25))
    
    # Set y-axis ticks from 0 to 1 with an interval of 0.1
    plt.yticks(np.linspace(0, 1, 11))
    plt.ylim(0, 1.1)
    
    # Save the plot
    plt.savefig(results_folder + file_name + ".png")
    plt.close()

In [39]:
from sklearn.metrics import make_scorer
from itertools import product

In [43]:
test_results_list = []
train_results_list = []

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [200],
    'learning_rate': [0.1, 0.3, 0.05],
    'max_depth': [3, 5],
    'subsample': [1.0, 0.8],
    'colsample_bytree': [1.0, 0.8],
    'alpha': [1.0],
    'reg_lambda': [2.0],
    'scale_pos_weight': [10.0, 15.0, 5.0]
}

# Generate all combinations of parameters
param_combinations = list(product(
    param_grid['n_estimators'],
    param_grid['learning_rate'],
    param_grid['max_depth'],
    param_grid['subsample'],
    param_grid['colsample_bytree'],
    param_grid['alpha'],
    param_grid['reg_lambda'],
    param_grid['scale_pos_weight']
))

kf = KFold(n_splits=3, shuffle=True, random_state=42)

best_param_path = results_folder+"Hyperparameters/"

thresholds = [0.5, 0.6, 0.7, 0.8, 0.9]

num_features = list(range(len(featboost_selected_features))) + [X_train.shape[1]]

for i in num_features:
    print(i)
    
    best_estimator = xgb.XGBClassifier()
    
    if i != X_train.shape[1]:

        relevant_features = featboost_selected_features[:i+1]
        X_train_subset = X_train[:, relevant_features]
        X_test_subset = X_test[:, relevant_features]
    else:
        X_train_subset = X_train
        X_test_subset = X_test

    best_score = -np.inf
    best_params = None

    # Manually iterate through each combination of hyperparameters
    count = 0
    for params in param_combinations:
        print("Param " + str(count) + "/" + str(len(param_combinations)))
        param_dict = {
            'n_estimators': params[0],
            'learning_rate': params[1],
            'max_depth': params[2],
            'subsample': params[3],
            'colsample_bytree': params[4],
            'alpha': params[5],
            'reg_lambda': params[6],
            'scale_pos_weight': params[7]
        }

        fold_scores = []

        for train_idx, val_idx in kf.split(X_train_subset):
            X_train_fold, X_val_fold = X_train_subset[train_idx], X_train_subset[val_idx]
            y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

            # Train model with the current set of parameters
            model = xgb.XGBClassifier(**param_dict)
            model.fit(X_train_fold, y_train_fold)

            # Predict probabilities for the validation set
            y_val_pred_probs = model.predict_proba(X_val_fold)[:, 1]

            # Compute AP@K for the validation set
            k =  int(X_val_fold.shape[0])  # Set k dynamically based on the validation set size
            fold_score = apk_binary(y_val_fold, y_val_pred_probs, k)
            fold_scores.append(fold_score)

        # Calculate the average score across all folds
        mean_fold_score = np.mean(fold_scores)

        # Update the best parameters if the current score is better
        if mean_fold_score > best_score:
            best_score = mean_fold_score
            best_params = param_dict
        
        count += 1

    print("Best Parameters:", best_params)
    print("Best Cross-Validated AP@K Score:", best_score)
    
    with open(best_param_path + 'model_' + str(i) + '.json', 'w') as f:
        json.dump(best_params, f)
    
    best_estimator.set_params(**best_params)
    best_estimator.fit(X_train_subset, y_train)
    
    # Evaluate on the training set
    predictions_train, pred_probs_train, pred_probs_all_train = predict_normal(best_estimator, X_train_subset)
    performance_results_train = calc_metrics_at_thresholds(i+1, pred_probs_train, y_train, thresholds, k=100)
    
    train_results_list.extend(performance_results_train)
    
    # Evaluate on the in distribution test set
    predictions_test, pred_probs_test, pred_probs_all_test = predict_normal(best_estimator, X_test_subset)
    plot_precision_vs_k(pred_probs_test, y_test, "Precision_at_k_test_" + str(i) + ".png", max_k=250)
    performance_results_test = calc_metrics_at_thresholds(i+1, pred_probs_test, y_test, thresholds, k=100)
    
    test_results_list.extend(performance_results_test)

0
Param 0/72
Param 1/72
Param 2/72
Param 3/72
Param 4/72
Param 5/72
Param 6/72
Param 7/72
Param 8/72
Param 9/72
Param 10/72
Param 11/72
Param 12/72
Param 13/72
Param 14/72
Param 15/72
Param 16/72
Param 17/72
Param 18/72
Param 19/72
Param 20/72
Param 21/72
Param 22/72
Param 23/72
Param 24/72
Param 25/72
Param 26/72
Param 27/72
Param 28/72
Param 29/72
Param 30/72
Param 31/72
Param 32/72
Param 33/72
Param 34/72
Param 35/72
Param 36/72
Param 37/72
Param 38/72
Param 39/72
Param 40/72
Param 41/72
Param 42/72
Param 43/72
Param 44/72
Param 45/72
Param 46/72
Param 47/72
Param 48/72
Param 49/72
Param 50/72
Param 51/72
Param 52/72
Param 53/72
Param 54/72
Param 55/72
Param 56/72
Param 57/72
Param 58/72
Param 59/72
Param 60/72
Param 61/72
Param 62/72
Param 63/72
Param 64/72
Param 65/72
Param 66/72
Param 67/72
Param 68/72
Param 69/72
Param 70/72
Param 71/72
Best Parameters: {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 3, 'subsample': 0.8, 'colsample_bytree': 1.0, 'alpha': 1.0, 'reg_lamb

/mnt/nfs/home/chhedah/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/mnt/nfs/home/chhedah/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


2
Param 0/72
Param 1/72
Param 2/72
Param 3/72
Param 4/72
Param 5/72
Param 6/72
Param 7/72
Param 8/72
Param 9/72
Param 10/72
Param 11/72
Param 12/72
Param 13/72
Param 14/72
Param 15/72
Param 16/72
Param 17/72
Param 18/72
Param 19/72
Param 20/72
Param 21/72
Param 22/72
Param 23/72
Param 24/72
Param 25/72
Param 26/72
Param 27/72
Param 28/72
Param 29/72
Param 30/72
Param 31/72
Param 32/72
Param 33/72
Param 34/72
Param 35/72
Param 36/72
Param 37/72
Param 38/72
Param 39/72
Param 40/72
Param 41/72
Param 42/72
Param 43/72
Param 44/72
Param 45/72
Param 46/72
Param 47/72
Param 48/72
Param 49/72
Param 50/72
Param 51/72
Param 52/72
Param 53/72
Param 54/72
Param 55/72
Param 56/72
Param 57/72
Param 58/72
Param 59/72
Param 60/72
Param 61/72
Param 62/72
Param 63/72
Param 64/72
Param 65/72
Param 66/72
Param 67/72
Param 68/72
Param 69/72
Param 70/72
Param 71/72
Best Parameters: {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 3, 'subsample': 1.0, 'colsample_bytree': 0.8, 'alpha': 1.0, 'reg_lamb

/mnt/nfs/home/chhedah/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/mnt/nfs/home/chhedah/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


3
Param 0/72
Param 1/72
Param 2/72
Param 3/72
Param 4/72
Param 5/72
Param 6/72
Param 7/72
Param 8/72
Param 9/72
Param 10/72
Param 11/72
Param 12/72
Param 13/72
Param 14/72
Param 15/72
Param 16/72
Param 17/72
Param 18/72
Param 19/72
Param 20/72
Param 21/72
Param 22/72
Param 23/72
Param 24/72
Param 25/72
Param 26/72
Param 27/72
Param 28/72
Param 29/72
Param 30/72
Param 31/72
Param 32/72
Param 33/72
Param 34/72
Param 35/72
Param 36/72
Param 37/72
Param 38/72
Param 39/72
Param 40/72
Param 41/72
Param 42/72
Param 43/72
Param 44/72
Param 45/72
Param 46/72
Param 47/72
Param 48/72
Param 49/72
Param 50/72
Param 51/72
Param 52/72
Param 53/72
Param 54/72
Param 55/72
Param 56/72
Param 57/72
Param 58/72
Param 59/72
Param 60/72
Param 61/72
Param 62/72
Param 63/72
Param 64/72
Param 65/72
Param 66/72
Param 67/72
Param 68/72
Param 69/72
Param 70/72
Param 71/72
Best Parameters: {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 3, 'subsample': 1.0, 'colsample_bytree': 0.8, 'alpha': 1.0, 'reg_lamb

/mnt/nfs/home/chhedah/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/mnt/nfs/home/chhedah/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


4
Param 0/72
Param 1/72
Param 2/72
Param 3/72
Param 4/72
Param 5/72
Param 6/72
Param 7/72
Param 8/72
Param 9/72
Param 10/72
Param 11/72
Param 12/72
Param 13/72
Param 14/72
Param 15/72
Param 16/72
Param 17/72
Param 18/72
Param 19/72
Param 20/72
Param 21/72
Param 22/72
Param 23/72
Param 24/72
Param 25/72
Param 26/72
Param 27/72
Param 28/72
Param 29/72
Param 30/72
Param 31/72
Param 32/72
Param 33/72
Param 34/72
Param 35/72
Param 36/72
Param 37/72
Param 38/72
Param 39/72
Param 40/72
Param 41/72
Param 42/72
Param 43/72
Param 44/72
Param 45/72
Param 46/72
Param 47/72
Param 48/72
Param 49/72
Param 50/72
Param 51/72
Param 52/72
Param 53/72
Param 54/72
Param 55/72
Param 56/72
Param 57/72
Param 58/72
Param 59/72
Param 60/72
Param 61/72
Param 62/72
Param 63/72
Param 64/72
Param 65/72
Param 66/72
Param 67/72
Param 68/72
Param 69/72
Param 70/72
Param 71/72
Best Parameters: {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 3, 'subsample': 1.0, 'colsample_bytree': 0.8, 'alpha': 1.0, 'reg_lamb

/mnt/nfs/home/chhedah/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/mnt/nfs/home/chhedah/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


6
Param 0/72
Param 1/72
Param 2/72
Param 3/72
Param 4/72
Param 5/72
Param 6/72
Param 7/72
Param 8/72
Param 9/72
Param 10/72
Param 11/72
Param 12/72
Param 13/72
Param 14/72
Param 15/72
Param 16/72
Param 17/72
Param 18/72
Param 19/72
Param 20/72
Param 21/72
Param 22/72
Param 23/72
Param 24/72
Param 25/72
Param 26/72
Param 27/72
Param 28/72
Param 29/72
Param 30/72
Param 31/72
Param 32/72
Param 33/72
Param 34/72
Param 35/72
Param 36/72
Param 37/72
Param 38/72
Param 39/72
Param 40/72
Param 41/72
Param 42/72
Param 43/72
Param 44/72
Param 45/72
Param 46/72
Param 47/72
Param 48/72
Param 49/72
Param 50/72
Param 51/72
Param 52/72
Param 53/72
Param 54/72
Param 55/72
Param 56/72
Param 57/72
Param 58/72
Param 59/72
Param 60/72
Param 61/72
Param 62/72
Param 63/72
Param 64/72
Param 65/72
Param 66/72
Param 67/72
Param 68/72
Param 69/72
Param 70/72
Param 71/72
Best Parameters: {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 3, 'subsample': 0.8, 'colsample_bytree': 0.8, 'alpha': 1.0, 'reg_lamb

In [101]:
test_results_df = pd.DataFrame(test_results_list)
train_results_df = pd.DataFrame(train_results_list)

In [103]:
test_results_df.to_csv(results_folder+"Test_results.csv", index=False)
train_results_df.to_csv(results_folder+"Train_results.csv", index=False)